# Ice Velocity Sample 
This script samples the 6-day and 12-day ITS_LIVE velocity pairs previously downloaded using the 'ice_velocity_download' notebook. Velocity is sampled at specifix X, Y locations specified within the code. 

In [1]:
# IMPORT REQUIRED MODULES [velocity_sample environment]
import os
import csv
from osgeo import gdal
import numpy as np
import datetime

Provide the directory path to the velocity GeoTIFFs and input the desired directory (including filename) of the output CSV. Provide the X, Y coordinates of your desired sampling location. 

In [2]:
# DEFINE THE REGION AND PROVIDE THE FILEPATH TO THE GEOTIFFS
folder_path = 'R:/JAKOBSHAVN/DATA/ice_surface_velocity/ITS_LIVE_6_DAY/ITS_LIVE_vel_EPSG3413_G0120_X-150000_Y-2250000'

# DEFINE THE OUTPUT CSV FILE LOCATION AND NAME
csv_file = 'R:/JAKOBSHAVN/DATA/ice_surface_velocity/ITS_LIVE_6_DAY/its_live_sampled_6_12_day.csv'

# LIST THE X AND Y CO-ORDINATES AT WHICH VELOCITY SHOULD BE EXTRACTED
coordinates = [(-180091.9495, -2278118.175),  
    (-176595.9169, -2281533.351),
    (-171733.1094, -2280782.72),
    (-166897.6833, -2279795.733),
    (-161907.8652, -2279370.56),
    (-157584.877, -2279370.56),
    (-153231.406, -2279370.56),]

The following script will output a CSV that includes the sampled velocity. For each value, the corresponding filename from which the velocity was sampled is given. From this filename, the start date, mid date and end date are also extracted.

In [3]:
# CREATE A CSV THAT WILL SAMPLE VELOCITY FROM EACH POINT, PROVIDING DETAILS OF THE CORRESPONDING FILENAME, START DATE, END DATE AND MID DATE
csv_headers = ['Filename', 'start_date', 'end_date', 'mid_date']
for i, (x, y) in enumerate(coordinates, start=1):
    csv_headers.append(f'POINT_{i}_VELOCITY')  # Custom column headers for each point's velocity
with open(csv_file, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(csv_headers)
    for filename in os.listdir(folder_path):
        if filename.endswith('.tif') or filename.endswith('.tiff'):
            file_path = os.path.join(folder_path, filename)
            date_parts = filename.split('_')
            start_date = date_parts[0]
            end_date = date_parts[1]
            start_date_obj = datetime.datetime.strptime(start_date, "%Y-%m-%d")
            end_date_obj = datetime.datetime.strptime(end_date, "%Y-%m-%d")
            mid_date_obj = start_date_obj + (end_date_obj - start_date_obj) / 2
            mid_date = mid_date_obj.strftime("%Y/%m/%d")
            dataset = gdal.Open(file_path, gdal.GA_ReadOnly)
            if dataset is not None:
                values = [filename, start_date, end_date, mid_date]
                for x, y in coordinates:
                    geotransform = dataset.GetGeoTransform()
                    x_pixel = int((x - geotransform[0]) / geotransform[1])
                    y_pixel = int((y - geotransform[3]) / geotransform[5])
                    band = dataset.GetRasterBand(1)
                    value = band.ReadAsArray(x_pixel, y_pixel, 1, 1)[0][0]
                    if np.isnan(value):
                        value = ''
                    values.append(value)
                csv_writer.writerow(values)
                dataset = None

print(f"Data extracted and saved to '{csv_file}'.")

Data extracted and saved to 'R:/JAKOBSHAVN/DATA/ice_surface_velocity/ITS_LIVE_6_DAY/its_live_sampled_6_12_day.csv'.
